### Task 1.1

In [16]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import pairwise_distances
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel

# Initialize ResNet feature extractor
feature_extractor = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3), pooling='avg')

# Function to extract features in batches to manage memory
def extract_features(images, batch_size=32):
    features_list = []
    for i in range(0, len(images), batch_size):
        batch_images = preprocess_input(images[i:i + batch_size])
        features = feature_extractor.predict(batch_images)
        features_list.append(features)
    return np.vstack(features_list)

class KernelLwPClassifier:
    def __init__(self, feature_dim, num_classes, gamma=None):
        self.prototypes = np.zeros((num_classes, feature_dim))  # Mean prototype for each class
        self.class_counts = np.zeros(num_classes)  # Track the total count of samples for each class
        self.gamma = gamma  # RBF kernel coefficient

    def fit(self, features, labels):
        """Initialize the prototypes with class means from the labeled dataset."""
        for i in range(self.prototypes.shape[0]):  # Iterate over the number of classes
            class_features = features[labels == i]
            if len(class_features) > 0:
                self.prototypes[i] = np.mean(class_features, axis=0)
                self.class_counts[i] = len(class_features)

    def _compute_kernel(self, X, Y):
        """Compute RBF kernel similarity between X and Y."""
        if self.gamma is None:
            self.gamma = 1.0 / X.shape[1]  # Default gamma: 1 / feature_dim
        return rbf_kernel(X, Y, gamma=self.gamma)

    def predict(self, features):
        """Predict class based on the prototype with the highest kernel similarity."""
        # Compute similarity between features and prototypes
        similarities = self._compute_kernel(features, self.prototypes)
        return np.argmax(similarities, axis=1)  # Return the class with the highest similarity score

    def update_prototypes(self, features, pseudo_labels):
        """Update prototypes incrementally based on high-confidence pseudo-labeled data."""
        for i in range(self.prototypes.shape[0]):
            class_features = features[pseudo_labels == i]
            if len(class_features) > 0:
                new_mean = np.mean(class_features, axis=0)
                total_count = self.class_counts[i] + len(class_features)
                self.prototypes[i] = (self.prototypes[i] * self.class_counts[i] + new_mean * len(class_features)) / total_count
                self.class_counts[i] = total_count

    def cross_validate_gamma(self, features, labels, gamma_values, cv_splits=5):
        """Find the best gamma value using cross-validation."""
        best_gamma = None
        best_accuracy = 0
        accuracies = []

        kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)

        for gamma in gamma_values:
            self.gamma = gamma  # Set gamma for this fold
            fold_accuracies = []

            for train_index, val_index in kf.split(features):
                # Split the data into training and validation sets
                X_train, X_val = features[train_index], features[val_index]
                y_train, y_val = labels[train_index], labels[val_index]

                # Fit the classifier on the training set
                self.fit(X_train, y_train)

                # Predict on the validation set
                y_pred = self.predict(X_val)

                # Compute accuracy for this fold
                fold_accuracies.append(accuracy_score(y_val, y_pred))

            # Average accuracy across folds for the current gamma
            mean_accuracy = np.mean(fold_accuracies)
            accuracies.append(mean_accuracy)

            if mean_accuracy > best_accuracy:
                best_accuracy = mean_accuracy
                best_gamma = gamma

        self.gamma = best_gamma  # Update gamma to the best value
        print(f"Best gamma: {best_gamma}, Accuracy: {best_accuracy}")
        return best_gamma, accuracies

# Pseudo-labeling function with confidence filtering
def pseudo_label_dataset(classifier, features, confidence_threshold=0.7):
    pseudo_labels = classifier.predict(features)
    distances = pairwise_distances(features, classifier.prototypes, metric='euclidean')
    confidences = 1 / (np.min(distances, axis=1) + 1e-5)
    confidences_normalized = (confidences - confidences.min()) / (confidences.max() - confidences.min())
    high_conf_indices = confidences_normalized >= confidence_threshold
    return features[high_conf_indices], pseudo_labels[high_conf_indices]

# Training loop across datasets D1 to D10
def train_incremental_lwp_classifier(num_classes, feature_dim, datasets_path):
    # classifier = IncrementalLwPClassifier(feature_dim, num_classes)
    t = torch.load('dataset/part_one_dataset/train_data/1_train_data.tar.pth') # datasets/part_one_dataset/train/
    print(t.keys()) # it will print data and targets

    # Define gamma candidates
    gamma_values = [0.0001, 0.001, 0.01, 0.1, 1]

    # Instantiate classifier
    classifier = KernelLwPClassifier(feature_dim, num_classes)

    # Cross-validation to find the best gamma
    features = extract_features(t["data"])  # Extract features from dataset
    labels = t["targets"]  # Get corresponding labels

    # Perform cross-validation
    best_gamma, accuracies = classifier.cross_validate_gamma(features, labels, gamma_values, cv_splits=5)

    print(f"Best gamma: {best_gamma}")
    print(f"Validation accuracies: {accuracies}")

    # Use the classifier with the best gamma for further training
    classifier.fit(features, labels)
    accuracy_matrix = np.zeros((10, 10))  # Accuracy matrix for tracking results

    for i in range(1, 11):
        # Load and process dataset
        print(f"Training model on pseudo-labeled dataset D{i}")
        data_path = f'{datasets_path}/train_data/{i}_train_data.tar.pth'
        dataset = torch.load(data_path)
        features = extract_features(dataset['data'])
        labels = dataset['targets'] if i == 1 else None

        # Initialize or update prototypes
        if i == 1:
            classifier.fit(features, labels)
        else:
            pseudo_features, pseudo_labels = pseudo_label_dataset(classifier, features)
            # classifier.update_prototypes_incrementally(pseudo_features, pseudo_labels)
            classifier.update_prototypes(pseudo_features, pseudo_labels)

        # Evaluate on held-out datasets
        for j in range(1, i + 1):
            eval_data_path = f'{datasets_path}/eval_data/{j}_eval_data.tar.pth'
            eval_dataset = torch.load(eval_data_path)
            test_features = extract_features(eval_dataset['data'])
            y_true = eval_dataset['targets']
            y_pred = classifier.predict(test_features)
            accuracy = accuracy_score(y_true, y_pred)
            accuracy_matrix[i - 1, j - 1] = accuracy
            print(f"Accuracy on D{j} after training on D{i}: {accuracy:.4f}")

    return classifier, accuracy_matrix

# Example usage
datasets_path = "dataset/part_one_dataset"
classifier, accuracy_matrix = train_incremental_lwp_classifier(10, 2048, datasets_path)


dict_keys(['data', 'targets'])
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/st

In [17]:
print("Final Accuracy Matrix:\n", accuracy_matrix)

Final Accuracy Matrix:
 [[0.486  0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.482  0.4904 0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.4756 0.4876 0.488  0.     0.     0.     0.     0.     0.     0.    ]
 [0.4772 0.488  0.4868 0.4984 0.     0.     0.     0.     0.     0.    ]
 [0.4752 0.4872 0.4868 0.498  0.4772 0.     0.     0.     0.     0.    ]
 [0.4732 0.4864 0.4844 0.4968 0.4736 0.4956 0.     0.     0.     0.    ]
 [0.4728 0.4856 0.4844 0.4956 0.4732 0.494  0.476  0.     0.     0.    ]
 [0.472  0.484  0.4836 0.4952 0.4736 0.4936 0.4768 0.4836 0.     0.    ]
 [0.472  0.484  0.4828 0.4948 0.4744 0.4932 0.4768 0.4836 0.4764 0.    ]
 [0.4716 0.4836 0.4836 0.494  0.4732 0.4936 0.4756 0.4832 0.4756 0.492 ]]


In [19]:
import pandas as pd
pd.DataFrame(accuracy_matrix, columns = list(range(1, 11)), index = list(range(1, 11)))

,1,2,3,4,5,6,7,8,9,10
1,0.4860,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
2,0.4820,0.4904,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
3,0.4756,0.4876,0.4880,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
4,0.4772,0.4880,0.4868,0.4984,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
5,0.4752,0.4872,0.4868,0.4980,0.4772,0.0000,0.0000,0.0000,0.0000,0.000
6,0.4732,0.4864,0.4844,0.4968,0.4736,0.4956,0.0000,0.0000,0.0000,0.000
7,0.4728,0.4856,0.4844,0.4956,0.4732,0.4940,0.4760,0.0000,0.0000,0.000
8,0.4720,0.4840,0.4836,0.4952,0.4736,0.4936,0.4768,0.4836,0.0000,0.000
9,0.4720,0.4840,0.4828,0.4948,0.4744,0.4932,0.4768,0.4836,0.4764,0.000
10,0.4716,0.4836,0.4836,0.4940,0.4732,0.4936,0.4756,0.4832,0.4756,0.492


### Task 1.2

In [9]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from sklearn.metrics import pairwise_distances
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf

# Initialize ResNet feature extractor
feature_extractor = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3), pooling='avg')

# Pseudo-labeling with confidence filtering
def pseudo_label_dataset(classifier, features, confidence_threshold=0.8):
    pseudo_labels = classifier.predict(features)
    distances = pairwise_distances(features, classifier.prototypes, metric='euclidean')
    confidences = 1 / (np.min(distances, axis=1) + 1e-5)
    confidences_normalized = (confidences - confidences.min()) / (confidences.max() - confidences.min())
    high_conf_indices = confidences_normalized >= confidence_threshold
    return features[high_conf_indices], pseudo_labels[high_conf_indices]

# Training loop for Task 1.2
def train_incremental_lwp_classifier(num_classes, feature_dim, datasets_path):
    # classifier2 = IncrementalLwPClassifier(feature_dim, num_classes, classifier.)
    accuracy_matrix = np.zeros((10, 20))  # For models f11 to f20 evaluated on D̂1 to D̂20

    # Initialize with model f10
    for i in range(10):
        print(f"Training model on pseudo-labeled dataset D{i+11}")
        data_path = f'{datasets_path}/train_data/{i + 1}_train_data.tar.pth'
        dataset = torch.load(data_path)
        features = extract_features(dataset['data'])
        # labels = dataset['targets'] if i == 10 else None

        # Update prototypes incrementally
        # if i == 10:
        #     classifier.fit(features, labels)
        # else:
        pseudo_features, pseudo_labels = pseudo_label_dataset(classifier, features)
        # classifier.update_prototypes_incrementally(pseudo_features, pseudo_labels)
        classifier.update_prototypes(pseudo_features, pseudo_labels)

        # Evaluate on held-out datasets
        for j in range(11 + i):
            if j<=9:
                eval_data_path = f'{datasets_path.replace("two", "one")}/eval_data/{j + 1}_eval_data.tar.pth'
            else:
                eval_data_path = f'{datasets_path}/eval_data/{j-10 + 1}_eval_data.tar.pth'
            eval_dataset = torch.load(eval_data_path)
            test_features = extract_features(eval_dataset['data'])
            y_true = eval_dataset['targets']
            y_pred = classifier.predict(test_features)
            accuracy = accuracy_score(y_true, y_pred)
            accuracy_matrix[i, j] = accuracy
            print(f"Accuracy on D̂{j + 1} after training on D{i + 11}: {accuracy:.4f}")

    return classifier, accuracy_matrix

# Example usage
datasets_path = "dataset/part_two_dataset"
classifier, accuracy_matrix = train_incremental_lwp_classifier(10, 2048, datasets_path)


Training model on pseudo-labeled dataset D11
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━

In [10]:
print("Final Accuracy Matrix:", accuracy_matrix)

Final Accuracy Matrix: [[0.47   0.4844 0.484  0.4948 0.472  0.4928 0.4748 0.48   0.4756 0.4912
  0.4288 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.4704 0.484  0.4848 0.4936 0.4724 0.4924 0.4744 0.4796 0.476  0.4904
  0.4284 0.3328 0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.47   0.4832 0.4848 0.494  0.472  0.494  0.4752 0.4788 0.4764 0.4908
  0.4276 0.3332 0.418  0.     0.     0.     0.     0.     0.     0.    ]
 [0.47   0.484  0.4868 0.4936 0.472  0.4924 0.4752 0.4788 0.476  0.4908
  0.428  0.3336 0.418  0.3588 0.     0.     0.     0.     0.     0.    ]
 [0.4704 0.4836 0.4868 0.4936 0.472  0.4932 0.4756 0.4792 0.4756 0.4908
  0.428  0.3336 0.418  0.3592 0.5072 0.     0.     0.     0.     0.    ]
 [0.4712 0.4836 0.486  0.4936 0.4724 0.494  0.4752 0.4792 0.4764 0.49
  0.4292 0.3336 0.4192 0.3588 0.5068 0.4024 0.     0.     0.     0.    ]
 [0.4712 0.4832 0.486  0.4932 0.4716 0.4936 0.4764 0.4788 0.4772 0.49
  0.4296 0.3332 0.42   0.3596 0.5068 0.

In [14]:
import pandas as pd
pd.DataFrame(accuracy_matrix, columns = list(range(1, 21)), index = list(range(1, 11)))

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,0.4700,0.4844,0.4840,0.4948,0.4720,0.4928,0.4748,0.4800,0.4756,0.4912,0.4288,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00
2,0.4704,0.4840,0.4848,0.4936,0.4724,0.4924,0.4744,0.4796,0.4760,0.4904,0.4284,0.3328,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00
3,0.4700,0.4832,0.4848,0.4940,0.4720,0.4940,0.4752,0.4788,0.4764,0.4908,0.4276,0.3332,0.4180,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00
4,0.4700,0.4840,0.4868,0.4936,0.4720,0.4924,0.4752,0.4788,0.4760,0.4908,0.4280,0.3336,0.4180,0.3588,0.0000,0.0000,0.0000,0.0000,0.0000,0.00
5,0.4704,0.4836,0.4868,0.4936,0.4720,0.4932,0.4756,0.4792,0.4756,0.4908,0.4280,0.3336,0.4180,0.3592,0.5072,0.0000,0.0000,0.0000,0.0000,0.00
6,0.4712,0.4836,0.4860,0.4936,0.4724,0.4940,0.4752,0.4792,0.4764,0.4900,0.4292,0.3336,0.4192,0.3588,0.5068,0.4024,0.0000,0.0000,0.0000,0.00
7,0.4712,0.4832,0.4860,0.4932,0.4716,0.4936,0.4764,0.4788,0.4772,0.4900,0.4296,0.3332,0.4200,0.3596,0.5068,0.4020,0.3720,0.0000,0.0000,0.00
8,0.4716,0.4832,0.4868,0.4932,0.4712,0.4928,0.4764,0.4780,0.4772,0.4900,0.4292,0.3332,0.4196,0.3592,0.5068,0.4020,0.3720,0.3872,0.0000,0.00
9,0.4716,0.4836,0.4864,0.4932,0.4716,0.4932,0.4760,0.4776,0.4768,0.4896,0.4292,0.3336,0.4188,0.3596,0.5072,0.4024,0.3716,0.3872,0.4296,0.00
10,0.4716,0.4840,0.4864,0.4932,0.4712,0.4928,0.4764,0.4776,0.4768,0.4900,0.4296,0.3336,0.4188,0.3600,0.5072,0.4020,0.3716,0.3864,0.4296,0.48
